In [1]:
# Upload NAIP satellite data, creating false color image, saving binary mask and
# computing NDVI for 7 geographic tiles spanning the city of Redlands

In [2]:
import os
import glob
import rasterio
import numpy as np
import pandas as pd

In [3]:
#Paths
tif_dir = 'EarthExplorer_NAIPData/'

# Recursively find all .tif files inside subfolders
tif_files = sorted(glob.glob(os.path.join(tif_dir, '**', '*.tif'), recursive=True))

# Optional: Print file paths to confirm
for tif in tif_files:
    print(tif)

EarthExplorer_NAIPData/m_3411763_ne_11_060_20220501/m_3411763_ne_11_060_20220501.tif
EarthExplorer_NAIPData/m_3411763_nw_11_060_20220501/m_3411763_nw_11_060_20220501.tif
EarthExplorer_NAIPData/m_3411763_se_11_060_20220501/m_3411763_se_11_060_20220501.tif
EarthExplorer_NAIPData/m_3411763_sw_11_060_20220501/m_3411763_sw_11_060_20220501.tif
EarthExplorer_NAIPData/m_3411764_ne_11_060_20220531/m_3411764_ne_11_060_20220531.tif
EarthExplorer_NAIPData/m_3411764_nw_11_060_20220501/m_3411764_nw_11_060_20220501.tif
EarthExplorer_NAIPData/m_3411764_se_11_060_20220531/m_3411764_se_11_060_20220531.tif
EarthExplorer_NAIPData/m_3411764_sw_11_060_20220501/m_3411764_sw_11_060_20220501.tif


In [4]:
# Output folder where you will save results
output_dir = 'get_binarymask_ndvi_outputs/'
os.makedirs(output_dir, exist_ok=True)

In [6]:
# To store metadata per tile
summary = []

# Loop through all NAIP TIFs
for tif_path in tif_files:
    tile_name = os.path.basename(tif_path)
    
    with rasterio.open(tif_path) as src:
        red = src.read(1).astype('float32')
        nir = src.read(4).astype('float32')
        ndvi = (nir - red) / (nir + red + 1e-6)
        
        # Binary vegetation mask (NDVI > 0.4)
        veg_mask = (ndvi > 0.4).astype('uint8')

        num_veg_pixels = veg_mask.sum()
        
        # Compute vegetation fraction
        #veg_fraction = veg_mask.sum() / veg_mask.size
        #print(f'{tile_name}: vegetation pixels = {num_veg_pixels}, veg_fraction = {veg_fraction:.3f}')
        
        # Save NDVI GeoTIFF
        ndvi_meta = src.meta.copy()
        ndvi_meta.update({'count': 1, 'dtype': 'float32'})
        ndvi_path = os.path.join(output_dir, f"{tile_name.replace('.tif', '')}_ndvi.tif")
        with rasterio.open(ndvi_path, 'w', **ndvi_meta) as dst:
            dst.write(ndvi, 1)
        
        # Save vegetation mask GeoTIFF
        mask_meta = src.meta.copy()
        mask_meta.update({'count': 1, 'dtype': 'uint8'})
        mask_path = os.path.join(output_dir, f"{tile_name.replace('.tif', '')}_vegmask.tif")
        with rasterio.open(mask_path, 'w', **mask_meta) as dst:
            dst.write(veg_mask, 1)



        # Store metadata
        summary.append({
            'tile': tile_name,
            'width': src.width,
            'height': src.height,
            'veg_fraction': veg_fraction,
            'mean_ndvi': float(ndvi.mean())
        })


m_3411763_ne_11_060_20220501.tif: vegetation pixels = 7385445, veg_fraction = 0.059
m_3411763_nw_11_060_20220501.tif: vegetation pixels = 2584644, veg_fraction = 0.021
m_3411763_se_11_060_20220501.tif: vegetation pixels = 19100346, veg_fraction = 0.152
m_3411763_sw_11_060_20220501.tif: vegetation pixels = 7246066, veg_fraction = 0.057
m_3411764_ne_11_060_20220531.tif: vegetation pixels = 12442365, veg_fraction = 0.099
m_3411764_nw_11_060_20220501.tif: vegetation pixels = 10081489, veg_fraction = 0.080
m_3411764_se_11_060_20220531.tif: vegetation pixels = 3907994, veg_fraction = 0.031
m_3411764_sw_11_060_20220501.tif: vegetation pixels = 5990911, veg_fraction = 0.048


In [7]:
# Save tile summary CSV
df = pd.DataFrame(summary)
df.to_csv(os.path.join(output_dir, 'tile_summary.csv'), index=False)

In [9]:
df.head(10)

,tile,width,height,veg_fraction,mean_ndvi
0,m_3411763_ne_11_060_20220501.tif,10300,12230,0.058629,0.007145
1,m_3411763_nw_11_060_20220501.tif,10300,12240,0.020501,-0.040034
2,m_3411763_se_11_060_20220501.tif,10300,12230,0.151627,0.077257
3,m_3411763_sw_11_060_20220501.tif,10310,12240,0.057420,0.020944
4,m_3411764_ne_11_060_20220531.tif,10280,12230,0.098965,0.090488
5,m_3411764_nw_11_060_20220501.tif,10290,12230,0.080109,0.060820
6,m_3411764_se_11_060_20220531.tif,10290,12230,0.031054,0.011118
7,m_3411764_sw_11_060_20220501.tif,10300,12230,0.047559,0.027642
